In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import shared_vars
import datetime
from datetime import timedelta, date
import requests


In [2]:
from sqlalchemy import create_engine
import pymysql

#DB 연결
def db_conn(host='db-dfmba.cnm8u4m2cbtp.ap-northeast-2.rds.amazonaws.com' , port=3306):
    engine = create_engine(f'mysql+pymysql://root:alexalex@{host}:{port}/stock_db')
    con = pymysql.connect(user = 'root', 
                        passwd ='alexalex',
                        host = f'{host}', 
                        db='stock_db', 
                        charset = 'utf8',
                        cursorclass = pymysql.cursors.DictCursor)
    mycursor = con.cursor()
    return engine, con, mycursor

In [5]:
data = { "pdno":["111", "222", "333", "444"], "price":[ 1, 2, 3, 4]}
df = pd.DataFrame(data)
df

,pdno,price
0,111,1
1,222,2
2,333,3
3,444,4


In [10]:
import FinanceDataReader as fdr
import pandas as pd

# 1. AMEX에 등록된 ETF Ticker 목록 가져오기
def get_amex_etf_tickers():
    amex_tickers = fdr.StockListing('AMEX')  # AMEX 상장 종목 전체 목록
    etf_tickers = amex_tickers[amex_tickers['Sector'] == 'ETF']  # ETF 필터링
    return etf_tickers

# 2. 특정 ETF의 주가 데이터 가져오기
def get_etf_price_data(ticker, start_date='2020-01-01', end_date='2025-01-01'):
    return fdr.DataReader(ticker, start_date, end_date)


In [12]:
amex_tickers = fdr.StockListing('AMEX')

100%|██████████| 316/316 [00:00<00:00, 750.15it/s]


In [16]:
amex_tickers[amex_tickers.Symbol=='SPY']

,Symbol,Name,IndustryCode,Industry


In [18]:
start_date='2024-12-01'
end_date='2025-01-01'

eft_data = fdr.DataReader('SPY', start_date, end_date)

HTTP Error 401: Unauthorized  - symbol "SPY" not found or invalid periods


In [46]:
import FinanceDataReader as fdr
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import yfinance as yf
from datetime import timedelta, date
import time
import requests
import yaml

with open('config.yaml', encoding='UTF-8') as f:
 _cfg = yaml.load(f, Loader=yaml.FullLoader)
DISCORD_WEBHOOK_URL = _cfg['DISCORD_WEBHOOK_URL']

def send_message(msg):
    """디스코드 메세지 전송"""
    now = datetime.datetime.now()
    message = {"content": f"[{now.strftime('%b.%d %H:%M')}] {str(msg)}"}
    requests.post(DISCORD_WEBHOOK_URL, data=message)
    print(message)

#DB 연결
def db_conn(host='db-dfmba.cnm8u4m2cbtp.ap-northeast-2.rds.amazonaws.com' , port=3306):
    engine = create_engine(f'mysql+pymysql://root:alexalex@{host}:{port}/stock_db')
    con = pymysql.connect(user = 'root', 
                        passwd ='alexalex',
                        host = f'{host}', 
                        db='stock_db', 
                        charset = 'utf8',
                        cursorclass = pymysql.cursors.DictCursor)
    mycursor = con.cursor()
    return engine, con, mycursor

send_message(f"[ETF Update] Start ")

# data = { 'Symbol':['SPY', 'ARKQ'], 'Name':['SnP 500', 'ARK Autonomouse Technology & Robotics']}
# df_etf = pd.DataFrame(data)
engine, con, mycursor = db_conn()
sql = "select * from us_etf_list where symbol in ('ARKQ', 'SPY')"
mycursor.execute(sql)
result = mycursor.fetchall()
con.close()
df_etf = pd.DataFrame(result)
df_etf['IndustryCode'] = ""
df_etf['Industry'] = ""
df_etf['Market'] = "AMEX"
df_etf['ins_date'] = date.today()

print(f'df_eft : {df_etf}')
engine, con, mycursor = db_conn()
df_etf.to_sql(name = 'us_stock_list', con=engine, if_exists='append', index=False, method="multi")
con.close()

df_list = []

for etf_ticker in df_etf.iterrows():
    print(f'etf_ticker:{etf_ticker}')
    # print(f'type:{type(etf_ticker[1]['Symbol'])}')
    symbol = etf_ticker[1]['Symbol']
    print(f'ticker : {symbol}')

    start_date='2024-12-22'
    end_date= (date.today() - timedelta(days=1)).strftime('%Y-%m-%d')
    print(f'start date : {start_date}, end date:{end_date}')
    etf_data = yf.download(symbol, start=start_date, end=end_date)
    etf_data['Symbol'] = symbol
    etf_data.reset_index(inplace=True)
    etf_data = etf_data[['Symbol', 'Date', 'Close', 'Adj Close', 'Volume']]
 
    df_list.append(etf_data)

df_list = pd.concat(df_list, ignore_index=True)
# df_list

engine, con, mycursor = db_conn()
df_list.to_sql(name = 'us_stock_price', con=engine, if_exists='append', index=False)
con.close()




[*********************100%%**********************]  1 of 1 completed

{'content': '[Jan.22 01:14] [ETF Update] Start '}
df_eft :   Symbol                                   Name IndustryCode Industry Market  \
0    SPY                                SnP 500                         AMEX   
1   ARKQ  ARK Autonomouse Technology & Robotics                         AMEX   

     ins_date  
0  2025-01-22  
1  2025-01-22  
etf_ticker:(0, Symbol                 SPY
Name               SnP 500
IndustryCode              
Industry                  
Market                AMEX
ins_date        2025-01-22
Name: 0, dtype: object)
ticker : SPY
start date : 2024-12-22, end date:2025-01-21
etf_ticker:(1, Symbol                                           ARKQ
Name            ARK Autonomouse Technology & Robotics
IndustryCode                                         
Industry                                             
Market                                           AMEX
ins_date                                   2025-01-22
Name: 1, dtype: object)
ticker : ARKQ
start date : 202


[*********************100%%**********************]  1 of 1 completed


In [20]:
etf_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,323.540009,324.890015,322.529999,324.869995,301.194977,59151200
2020-01-03,321.160004,323.640015,321.100006,322.410004,298.914154,77709700
2020-01-06,320.489990,323.730011,320.359985,323.640015,300.054535,55653900
2020-01-07,323.019989,323.540009,322.239990,322.730011,299.210907,40496400
2020-01-08,322.940002,325.779999,322.670013,324.450012,300.805481,68296000
...,...,...,...,...,...,...
2024-12-24,596.059998,601.340027,595.469971,601.299988,601.299988,33160100
2024-12-26,599.500000,602.479980,598.080017,601.340027,601.340027,41219100
2024-12-27,597.539978,597.780029,590.760010,595.010010,595.010010,64969300


In [ ]:

# 실행
# AMEX에 등록된 ETF Ticker 확인
amex_etfs = get_amex_etf_tickers()

# ETF Ticker 출력 (상위 10개)
print("AMEX ETF Tickers:")
print(amex_etfs[['Symbol', 'Name']].head(10))

# 특정 ETF 주가 데이터 확인 (예: SPY)
ticker = 'SPY'  # S&P 500 ETF로 유명한 SPDR
etf_data = get_etf_price_data(ticker)

# ETF 주가 데이터 출력
print(f"\n{ticker} 주가 데이터:")
print(etf_data.head())

# CSV로 저장 (필요 시)
etf_data.to_csv(f"{ticker}_price_data.csv")


In [9]:
list = ["111", "333"]

df[~df.pdno.isin(list)]

,pdno,price
1,222,2
3,444,4


In [4]:
#DB 에서 보유 주식의 최고 수익율, 수익금액 조회 
engine, con, mycursor = db_conn()
sql = "select * from holding_stock_details"
mycursor.execute(sql)
result = mycursor.fetchall()
con.close()
df_holding_stock_details = pd.DataFrame(result)

In [5]:
df_holding_stock_details2 =df_holding_stock_details

In [25]:
# df_holding_stock_details.loc[:, 'test2'] = [True, False, True]
df_holding_stock_details.loc[:, 'test2'] = df_holding_stock_details['test']
df_holding_stock_details

,pdno,prdt_name,init_dt,prpr,evlu_amt,evlu_pfls_amt,evlu_pfls_rt,test,test2
0,005930,삼성전자,2024-12-30,53500,642000,100001,-2.90,True,True
1,071050,한국금융지주,2024-12-30,71800,502600,400001,1.84,False,False
2,360200,ACE 미국S&P500,2024-12-30,22105,22105,2000001,0.98,True,True


In [ ]:
df_holding_stock_details[]

In [7]:
df_holding_stock_details2.loc[df_holding_stock_details.pdno == '005930', 'evlu_pfls_amt'] = 100001
df_holding_stock_details2.loc[df_holding_stock_details.pdno == '071050', 'evlu_pfls_amt'] = 400001
df_holding_stock_details2.loc[df_holding_stock_details.pdno == '360200', 'evlu_pfls_amt'] = 2000001


In [63]:
df_merged = df_holding_stock_details2.merge(df_holding_stock_details,how='outer', on='pdno', suffixes=('_a', '_b'))

df_max_profit_for_update = df_merged[df_merged['evlu_pfls_amt_a'].astype(float) > df_merged['evlu_pfls_amt_b'].astype(float)][['pdno', 'evlu_pfls_amt_a']]

df_max_profit_for_update


,pdno,evlu_pfls_amt_a
0,005930,100001
1,071050,400001
2,360200,2000001


In [65]:
data = { 'amount':[ 100000, 200000, 400000, 700000, 1100000, 1600000, 2000000, 3000000, 5000000, 10000000, 9999999999], 
        'buy_ratio': [0, 50, 40, 30, 20, 20, 20, 20, 20, 30, 30]}
df_up_buy_table = pd.DataFrame(data)

def get_tier_ratio( amount, tier_t ):

        for index, row in tier_t.iterrows():
                
                # print(index)
                tier = row['amount']
                ratio = row['buy_ratio']
                # print(f'in get_tier_ratio() : {tier}, {ratio}, {amount}')

                if (tier >= amount):
                        return ratio , index +1
        
        return 0

In [ ]:
def get_tier_ratio_wrapper( row, tier_t ):

        for index, row in tier_t.iterrows():
                
                return get_tier_ratio(row['evlu_pfls_amt_a'], tier_t)
        

In [74]:
type(df_merged['evlu_pfls_amt_a'].apply(lambda x : get_tier_ratio(x, df_up_buy_table)))


TypeError: Series.apply() missing 1 required positional argument: 'func'

In [86]:
df_merged['profit_index_a'] = df_merged['evlu_pfls_amt_a'].apply(lambda x : get_tier_ratio(x, df_up_buy_table)).apply(lambda x: x[1]).tolist()
df_merged['profit_index_b'] = df_merged['evlu_pfls_amt_b'].apply(lambda x : get_tier_ratio(x, df_up_buy_table)).apply(lambda x: x[1]).tolist()
df_merged['buy_on_up_flag'] = df_merged['profit_index_a'] > df_merged['profit_index_b']
df_merged

,pdno,prdt_name_a,init_dt_a,prpr_a,evlu_amt_a,evlu_pfls_amt_a,evlu_pfls_rt_a,prdt_name_b,init_dt_b,prpr_b,evlu_amt_b,evlu_pfls_amt_b,evlu_pfls_rt_b,profit_index_a,profit_index_b,buy_on_up_flag
0,005930,삼성전자,2024-12-30,53500,642000,100001,-2.90,삼성전자,2024-12-30,53500,642000,0,-2.90,2,1,True
1,071050,한국금융지주,2024-12-30,71800,502600,400001,1.84,한국금융지주,2024-12-30,71800,502600,9800,1.84,4,1,True
2,360200,ACE 미국S&P500,2024-12-30,22105,22105,2000001,0.98,ACE 미국S&P500,2024-12-30,22105,22105,220,0.98,8,1,True


In [10]:
def send_message(msg):
    """디스코드 메세지 전송"""
    now = datetime.datetime.now()
    message = {"content": f"[{now.strftime('%b.%d %H:%M')}] {str(msg)}"}
    requests.post(shared_vars.DISCORD_WEBHOOK_URL, data=message)
    print(message)

In [12]:
def update_holding_stock_details():
    send_message(f"[최대 이익금액 업데이트]")
    """주식 잔고조회"""
    PATH = "uapi/domestic-stock/v1/trading/inquire-balance"
    URL = f"{shared_vars.URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
        "authorization":f"Bearer {shared_vars.ACCESS_TOKEN}",
        "appKey":shared_vars.APP_KEY,
        "appSecret":shared_vars.APP_SECRET,
        "tr_id":"TTTC8434R",
        "custtype":"P",
    }    
    params = {
        "CANO": shared_vars.CANO,
        "ACNT_PRDT_CD": shared_vars.ACNT_PRDT_CD,
        "AFHR_FLPR_YN": "N",
        "OFL_YN": "",
        "INQR_DVSN": "02",
        "UNPR_DVSN": "01",
        "FUND_STTL_ICLD_YN": "N",
        "FNCG_AMT_AUTO_RDPT_YN": "N",
        "PRCS_DVSN": "01",
        "CTX_AREA_FK100": "",
        "CTX_AREA_NK100": ""
    }
    res = requests.get(URL, headers=headers, params=params)
    curr_account_stock_list = res.json()['output1']    
    df_curr_account_stock = pd.DataFrame(curr_account_stock_list)
    df_curr_account_stock = df_curr_account_stock[['pdno','prdt_name','prpr','evlu_amt','evlu_pfls_amt','evlu_pfls_rt']]

    # Test code
    df_curr_account_stock = df_holding_stock_details2 
    # Test Code

    # df_curr_account_stock.loc[df_curr_account_stock.pdno == '005930','evlu_pfls_amt'] = -30006
    # df_curr_account_stock.loc[df_curr_account_stock.pdno == '071050','evlu_pfls_amt'] = -30002
    # print(f'df_curr_account_stock : {df_curr_account_stock}')

    holding_stock_tbl_col_list= ['pdno','prdt_name','init_dt','prpr','evlu_amt','evlu_pfls_amt','evlu_pfls_rt']

    df_update_stock_list = pd.DataFrame(columns=holding_stock_tbl_col_list)
    df_update_stock_list_empty = df_update_stock_list  

    # 계좌에 주식이 없으면 빈 데이터프레임 리턴하고 종료 
    if len(df_curr_account_stock) == 0:
        return df_update_stock_list

    try: 
        #DB 에서 보유 주식의 최고 수익율, 수익금액 조회 
        engine, con, mycursor = db_conn()
        sql = "select * from holding_stock_details"
        mycursor.execute(sql)
        result = mycursor.fetchall()
        con.close()
        df_holding_stock_details = pd.DataFrame(result)
    except Exception as e: 
        
        if e.args[0] == 1146:
            print('Table does not exist')
            
            df_curr_account_stock['init_dt'] = date.today()
            df_curr_account_stock = df_curr_account_stock[holding_stock_tbl_col_list]
            engine, con, mycursor = db_conn()
            df_curr_account_stock.to_sql(name = 'holding_stock_details', con=engine, if_exists='replace', index=False)
            con.close()
            df_update_stock_list = df_curr_account_stock
            send_message(f"[최대 이익금액 업데이트] {len(df_update_stock_list)} 종목 업데이트 됨")

            return df_update_stock_list_empty

    df_merged = df_curr_account_stock.merge(df_holding_stock_details,how='outer', on='pdno', suffixes=('_a', '_b'))

    df_merged['profit_index_a'] = df_merged['evlu_pfls_amt_a'].apply(lambda x : get_tier_ratio(x, df_up_buy_table)).apply(lambda x: x[1]).tolist()
    df_merged['profit_index_b'] = df_merged['evlu_pfls_amt_b'].apply(lambda x : get_tier_ratio(x, df_up_buy_table)).apply(lambda x: x[1]).tolist()
    df_merged['buy_on_up_flag'] = df_merged['profit_index_a'] > df_merged['profit_index_b']

    # print(f'merged : {df_merged}')
    # print(f'df_curr_account_stock desc : {df_curr_account_stock.dtypes}')
    # print(f'df_holding_stock_details desc : {df_holding_stock_details.dtypes}')
    print(f'merged desc : {df_merged.dtypes}')

    # 현재가 기준 이익 금액이 holding_stock_details 테이블의 이익금액 보다 큰 종목과 이익금액 조회
    df_max_profit_for_update = df_merged[df_merged['evlu_pfls_amt_a'].astype(float) > df_merged['evlu_pfls_amt_b'].astype(float)][['pdno', 'evlu_pfls_amt_a', 'buy_on_up_flag']]

    df_max_profit_for_update

    # 결과 출력
    print(f'df_max_profit_for_update: {df_max_profit_for_update}')

    engine, con, mycursor = db_conn()
    # 개별 종목의 holding_stock_details 의 최고 수익 금액 필드 업데이트 
    for index in df_max_profit_for_update.index:
        print(f'for {index} index')
        sql = """
        UPDATE holding_stock_details
        SET evlu_pfls_amt = %s
        WHERE pdno = %s
        """
        
        # 업데이트할 값
        evlu_pfls_amt = df_max_profit_for_update.loc[index, 'evlu_pfls_amt_a']
        pdno = df_max_profit_for_update.loc[index, 'pdno']
        
        # print(sql % (evlu_pfls_amt, pdno))
        # 쿼리 실행
        mycursor.execute(sql, (evlu_pfls_amt, pdno))
        
        # 변경 사항 커밋
        con.commit()
    con.close()

    print(f'len(df_max_profit_for_update) : {len(df_max_profit_for_update)}')

    # 최대이익 증가 종목에 대해 init_dt 오늘자로 update
    if len(df_max_profit_for_update) > 0: 
        df_update_stock_list = df_curr_account_stock[df_curr_account_stock['pdno'].isin(df_max_profit_for_update.pdno)]
        df_update_stock_list.loc[:,'init_dt'] = date.today()
        df_update_stock_list.loc[:,'buy_on_up_flag'] = df_max_profit_for_update['buy_on_up_flag']
        df_update_stock_list = df_update_stock_list[holding_stock_tbl_col_list]
        print(f'df_update_stock_list : {df_update_stock_list}')

    # 신규 편입된 종목에 대해서 holding_stock_details 에 레코드 insert
    for stock_code in df_merged[df_merged['evlu_pfls_amt_b'].isna()].pdno  :
        print(stock_code)
        dt_temp = df_curr_account_stock[df_curr_account_stock.pdno == stock_code]
        dt_temp.loc[:,'init_dt'] = date.today()
        dt_temp = dt_temp[holding_stock_tbl_col_list]
        engine, con, mycursor = db_conn()
        dt_temp.to_sql(name = 'holding_stock_details', con=engine, if_exists='append', index=False)
        con.close()

        df_update_stock_list = pd.concat([df_update_stock_list,dt_temp] )

    print(f'df_update_stock_list : {df_update_stock_list}')
    
    send_message(f"[최대 이익금액 업데이트] {len(df_update_stock_list)} 종목 업데이트 됨")
    
    return df_update_stock_list

# 기간별 주식 수익률 & 수익률 표준편차 계산 함수
def get_stock_return(stock_code, start_date, end_date):

    engine, con, mycursor = db_conn() 

    query = f"""
        SELECT code, date, close
        FROM kr_stock_price
        WHERE code = '{stock_code}' AND Date BETWEEN STR_TO_DATE('{start_date}', '%Y%m%d') AND STR_TO_DATE('{end_date}', '%Y%m%d')
    """
    mycursor.execute(query)
    result = mycursor.fetchall()
    df_stock_price = pd.DataFrame(result)
    con.close()    

    if df_stock_price.empty:
        return {
            'code': stock_code,
            'start_date': start_date,
            'end_date': end_date,
            'ret': None,
            'daily_std': None
        }

    # 날짜 순으로 정렬
    df_stock_price.sort_values('date', inplace=True)

    # 상승률 계산
    start_price = df_stock_price.iloc[0]['close']
    end_price = df_stock_price.iloc[-1]['close']
    price_change_rate = ((end_price - start_price) / start_price) * 100

    # 일별 주가 변동 표준편차 계산
    daily_price_std = df_stock_price['close'].pct_change().std()

    return {
        'code': stock_code,
        'start_date': start_date,
        'end_date': end_date,
        'ret': round(price_change_rate, 4),
        'daily_std': round(daily_price_std, 5)
    }

def get_tier_ratio( amount, tier_t ):

        for index, row in tier_t.iterrows():
                
                # print(index)
                tier = row['amount']
                ratio = row['buy_ratio']
                # print(f'in get_tier_ratio() : {tier}, {ratio}, {amount}')

                if (int(tier) >= int(amount)):
                        return ratio , index +1
        
        return 0
                
def buy_on_profit(stock_for_update, up_buy_t):

        buy_count = 0

        for index, row in stock_for_update.iterrows():
                print(f'======== ratio check ===========')
                print(f'pdno : {row.pdno}')
                profit_amt = row.evlu_pfls_amt
                print(f'profit_amt: {profit_amt}')
                ratio, index = get_tier_ratio(profit_amt, up_buy_t)
                print(f'ratio : {ratio}')
                print(f'index : {index}')
                print(f'evlu_amt : {row.evlu_amt }')
                print(f'ratio/100 : {ratio/100}')
                buy_amt = int(int(row.evlu_amt) * (ratio/100))
                print(f'buy amount : {buy_amt}')
                buy_qty = buy_amt // int(row.prpr)
                print(f'buy quantaty : {buy_qty}')

                if buy(row.pdno, row.prpr, buy_qty, '00'):
                    buy_count = buy_count + 1
        
        send_message(f"[최대이익증가 매수] {buy_count} 종목 추가 매수됨")
        return buy_count

In [21]:
import yaml

with open('config.yaml', encoding='UTF-8') as f:
    _cfg = yaml.load(f, Loader=yaml.FullLoader)
APP_KEY = _cfg['APP_KEY']
APP_SECRET = _cfg['APP_SECRET']
ACCESS_TOKEN = ""
CANO = _cfg['CANO']
ACNT_PRDT_CD = _cfg['ACNT_PRDT_CD']
DISCORD_WEBHOOK_URL = _cfg['DISCORD_WEBHOOK_URL']
URL_BASE = _cfg['URL_BASE']
HTS_ID = _cfg['HTS_ID']

# 공용 변수 설정 
shared_vars.URL_BASE = URL_BASE
shared_vars.APP_KEY = APP_KEY
shared_vars.APP_SECRET = APP_SECRET
shared_vars.HTS_ID = HTS_ID
shared_vars.DISCORD_WEBHOOK_URL = DISCORD_WEBHOOK_URL
shared_vars.CANO = CANO
shared_vars.ACNT_PRDT_CD = ACNT_PRDT_CD

In [48]:
engine, con, mycursor = db_conn()
sql = "select * from us_stock_list where symbol in ('ARKQ', 'SPY')"
mycursor.execute(sql)
result = mycursor.fetchall()
con.close()
df_etf = pd.DataFrame(result)

In [56]:
df_etf = pd.DataFrame(result)
df_etf = df_etf[df_etf.Market=='AMEX']
df_etf = df_etf[['Symbol', 'Name', 'Market']]
df_etf

,Symbol,Name,Market
2,SPY,SnP 500,AMEX
3,ARKQ,ARK Autonomouse Technology & Robotics,AMEX


In [59]:
engine, con, mycursor = db_conn()
df_etf.to_sql(name = 'us_etf_list', con=engine, if_exists='append', index=False)
con.close()

In [60]:
engine, con, mycursor = db_conn()
sql = "select * from us_etf_list where symbol in ('ARKQ', 'SPY')"
mycursor.execute(sql)
result = mycursor.fetchall()
con.close()
df_etf = pd.DataFrame(result)
df_etf

,Symbol,Name,Market
0,SPY,SnP 500,AMEX
1,ARKQ,ARK Autonomouse Technology & Robotics,AMEX


In [61]:
df_etf['IndustryCode'] = ""
df_etf['Industry'] = ""
df_etf['Market'] = "AMEX"
df_etf['ins_date'] = date.today()

df_etf

,Symbol,Name,Market,IndustryCode,Industry,ins_date
0,SPY,SnP 500,AMEX,,,2025-01-22
1,ARKQ,ARK Autonomouse Technology & Robotics,AMEX,,,2025-01-22


In [82]:
a_list > b_list

True

In [77]:
df_merged['evlu_pfls_amt_b'].apply(lambda x : get_tier_ratio(x, df_up_buy_table))

TypeError: '>=' not supported between instances of 'numpy.ndarray' and 'str'

In [85]:
def get_tier_ratio( amount, tier_t ):

        for index, row in tier_t.iterrows():
                
                # print(f'index:{index}')
                tier = row['amount']
                ratio = row['buy_ratio']
                # print(f'in get_tier_ratio() : {tier}, {ratio}, {amount}')

                if (int(tier) >= int(amount)):
                        return ratio , index +1
        
        return 0

df_merged['evlu_pfls_amt_b'].apply(lambda x : get_tier_ratio(x, df_up_buy_table))

0    (0, 1)
1    (0, 1)
2    (0, 1)
Name: evlu_pfls_amt_b, dtype: object

In [55]:


data = { 'amount':[ 100000, 200000, 400000, 700000, 1100000, 1600000, 2000000, 3000000, 5000000, 10000000, 9999999999], 
        'buy_ratio': [0, 50, 40, 30, 20, 20, 20, 20, 20, 30, 30]}
df_up_buy_table = pd.DataFrame(data)

def get_tier_ratio( amount, tier_t ):

        for index, row in tier_t.iterrows():
                
                # print(index)
                tier = row['amount']
                ratio = row['buy_ratio']
                # print(f'in get_tier_ratio() : {tier}, {ratio}, {amount}')

                if (tier >= amount):
                        return ratio , index +1
        
        return 0
                
def buy_on_profit(stock_for_update, up_buy_t):

        for index, row in stock_for_update.iterrows():
                print(f'======== ratio check ===========')
                print(f'pdno : {row.pdno}')
                profit_amt = row.evlu_pfls_amt
                print(f'profit_amt: {profit_amt}')
                ratio, index = get_tier_ratio(profit_amt, up_buy_t)
                print(f'ratio : {ratio}')
                print(f'index : {index}')
                print(f'evlu_amt : {row.evlu_amt }')
                print(f'ratio/100 : {ratio/100}')
                buy_amt = int(int(row.evlu_amt) * (ratio/100))
                print(f'buy amount : {buy_amt}')
                buy_qty = buy_amt // int(row.prpr)
                print(f'buy quantaty : {buy_qty}')
                buy(row.pdno, row.prpr, buy_qty, '00')


buy_on_profit(df_holding_stock_details, df_up_buy_table)

    

======== ratio check ===========
pdno : 005930
profit_amt: 100001
ratio : 50
index : 2
evlu_amt : 642000
ratio/100 : 0.5
buy amount : 321000
buy quantaty : 6
======== ratio check ===========
pdno : 071050
profit_amt: 400001
ratio : 30
index : 4
evlu_amt : 502600
ratio/100 : 0.3
buy amount : 150780
buy quantaty : 2
======== ratio check ===========
pdno : 360200
profit_amt: 2000001
ratio : 20
index : 8
evlu_amt : 22105
ratio/100 : 0.2
buy amount : 4421
buy quantaty : 0


In [18]:
data = { 'amount':[ 100000, 200000, 400000, 700000, 1100000, 1600000, 2000000, 3000000, 5000000, 10000000], 
        'buy_ratio': [25, 25, 30, 30, 20, 20, 20, 20, 20, 30]}
df_up_buy_table = pd.DataFrame(data)
df_up_buy_table


,amount,buy_ratio
0,100000,25
1,200000,25
2,400000,30
3,700000,30
4,1100000,20
5,1600000,20
6,2000000,20
7,3000000,20
8,5000000,20
9,10000000,30


In [6]:
data = { 'a': [1,2,4], 'b':['a', 'b', 'c']}
df_data =  pd.DataFrame(data)
df_data

,a,b
0,1,a
1,2,b
2,4,c
